In [13]:
# -*- coding: utf-8 -*-
import os
import zipfile
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import tensorflow as tf
from keras_preprocessing import image
from keras_preprocessing.image import ImageDataGenerator
import numpy as np

In [16]:
class Loading():
    def __init__(self):
        local_zip = 'rps.zip'
        zip_ref = zipfile.ZipFile(local_zip, 'r')
        zip_ref.extractall('train_data/')
        zip_ref.close()
        
        local_zip = 'rps-validation.zip'
        zip_ref = zipfile.ZipFile(local_zip, 'r')
        zip_ref.extractall('test_data/')
        zip_ref.close()
        
        local_zip = 'rps-test-set.zip'
        zip_ref = zipfile.ZipFile(local_zip, 'r')
        zip_ref.extractall('test_data/')
        zip_ref.close()
        
        self.rock_dir = os.path.join('train_data/rock')
        self.paper_dir = os.path.join('train_data/paper')
        self.scissors_dir = os.path.join('train_data/scissors')
        
        print('total training rock images:', len(os.listdir(rock_dir)))
        print('total training paper images:', len(os.listdir(paper_dir)))
        print('total training scissors images:', len(os.listdir(scissors_dir)))
        
        self.rock_files = os.listdir(rock_dir)
        print(rock_files[:10])
        
        self.paper_files = os.listdir(paper_dir)
        print(paper_files[:10])
        
        self.scissors_files = os.listdir(scissors_dir)
        print(scissors_files[:10])
        
        
        self.next_rock = None
        self.next_paper = None
        self.next_scissors = None
        
    def show_images(self):
        pic_index = 2
        
        self.next_rock = [os.path.join(self.rock_dir, fname) 
                        for fname in self.rock_files[pic_index-2:pic_index]]
        self.next_paper = [os.path.join(self.paper_dir, fname) 
                        for fname in self.paper_files[pic_index-2:pic_index]]
        self.next_scissors = [os.path.join(self.scissors_dir, fname) 
                        for fname in self.scissors_files[pic_index-2:pic_index]]
        
        for i, img_path in enumerate(self.next_rock+self.next_paper+self.next_scissors):
          #print(img_path)
          img = mpimg.imread(img_path)
          plt.imshow(img)
          plt.axis('Off')
          plt.show()
        

In [17]:
load_data_obj = Loading()

NameError: name 'rock_dir' is not defined

In [10]:
load_data_obj.show_images()

NameError: name 'rock_files' is not defined

In [ ]:
class Preprocessig(object):
    def __init__(self):
        self.model = None
        self.train_generator = None
        self.train_generator = None     
        
    def image_augmentation(self):
        
        TRAINING_DIR = "/daa/rps/"
        training_datagen = ImageDataGenerator(
            rescale = 1./255,
            rotation_range=40,
            width_shift_range=0.2,
            height_shift_range=0.2,
            shear_range=0.2,
            zoom_range=0.2,
            horizontal_flip=True,
            fill_mode='nearest')
        
        self.train_generator = training_datagen.flow_from_directory(
            TRAINING_DIR,
            target_size=(150,150),
            class_mode='categorical'
        )
        
        VALIDATION_DIR = "/data/rps-test-set/"
        validation_datagen = ImageDataGenerator(rescale = 1./255)
        self.validation_generator = validation_datagen.flow_from_directory(
            VALIDATION_DIR,
            target_size=(150,150),
            class_mode='categorical')

In [ ]:
preprocess_obj = Preprocessing()
preprocess_obj

In [ ]:
class DesignModel():
    def __init__(self):
        self.model = None
        self.history = None
        self.acc = None
        self.val_acc = None
        self.loss = None
        self.val_loss = None 
        
    def create_model(self):

        self.model = tf.keras.models.Sequential()
        
            tf.keras.layers.Conv2D(64, (3,3), activation='relu', input_shape=(150, 150, 3)),
            tf.keras.layers.MaxPooling2D(2, 2),
            # The second convolution
            tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
            tf.keras.layers.MaxPooling2D(2,2),
            # The third convolution
            tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
            tf.keras.layers.MaxPooling2D(2,2),
            # The fourth convolution
            tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
            tf.keras.layers.MaxPooling2D(2,2),
            # Flatten the results to feed into a DNN
            tf.keras.layers.Flatten(),
            tf.keras.layers.Dropout(0.5),
            # 512 neuron hidden layer
            tf.keras.layers.Dense(512, activation='relu'),
            tf.keras.layers.Dense(3, activation='softmax')
        ])
        
        
        self.model.summary()
        
        self.model.compile(loss = 'categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
        
        self.history = self.model.fit_generator(self.train_generator, epochs=25, validation_data = self.validation_generator, verbose = 1)

        self.model.save("rps.h5")

    def get_mattrics(self):
        
        self.acc = self.history.self.history['acc']
        self.val_acc = self.history.self.history['val_acc']
        self.loss = self.history.self.history['loss']
        self.val_loss = self.history.self.history['val_loss']

    def plot_accuracy(self):
        epochs = range(len(self.acc))
        
        plt.plot(epochs, self.acc, 'r', label='Training accuracy')
        plt.plot(epochs, self.val_acc, 'b', label='Validation accuracy')
        plt.title('Training and validation accuracy')
        plt.legend(loc=0)
        plt.figure()
        
        
        plt.show()
        
    def predict(self,path):
      img = image.load_img(path, target_size=(150, 150))
      x = image.img_to_array(img)
      x = np.expand_dims(x, axis=0)
    
      images = np.vstack([x])
      classes = self.model.predict(images, batch_size=10)
      print(img)
      print(classes)
          

In [3]:
rpsd = RockPaperScissorDetection()
rpsd.preprocessing()
rpsd.image_augmentation()
rpsd.create_model()

total training rock images: 840
total training paper images: 840
total training scissors images: 840
['rock01-108.png', 'rock03-112.png', 'rock06ck02-084.png', 'rock02-002.png', 'rock03-086.png', 'rock03-102.png', 'rock07-k03-009.png', 'rock04-071.png', 'rock04-036.png', 'rock06ck02-049.png']
['paper04-087.png', 'paper03-025.png', 'paper07-036.png', 'paper01-024.png', 'paper04-034.png', 'paper04-080.png', 'paper01-011.png', 'paper05-118.png', 'paper05-038.png', 'paper04-026.png']
['scissors01-086.png', 'scissors04-062.png', 'scissors03-065.png', 'testscissors01-095.png', 'scissors01-110.png', 'testscissors02-044.png', 'scissors03-060.png', 'testscissors02-039.png', 'testscissors01-115.png', 'scissors03-087.png']


FileNotFoundError: [Errno 2] No such file or directory: '/tmp/rps/'